# Chemical Kinetics

This code is to help us visualize the solution of nonlinear linear equations. <br>

This tutorial can be deployed in <a target="_blank" href="https://colab.research.google.com/github/ChemAI-Lab/Math4Chem/blob/main/website/Lecture_Notes/Coding/chem_kinetics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
import numpy as np

Example from [link](https://flexbooks.ck12.org/cbook/ck-12-chemistry-flexbook-2.0/section/18.10/primary/lesson/determining-the-rate-law-from-experimental-data-chem/)

| Experiment | [NO] | [H<sub>2</sub>] | Initial Rate [M/s]
| :---: | :---: | :---: | :---: |
| 1 | 0.005 | 0.002 | -11.29 |
| 2 | 0.01 | 0.002 | -9.90 |
| 3 | 0.01 | 0.004 | -9.21 |

Each of this experiment can be represented using the following rate law,
$$
r = k [NO]^{\alpha} [H_2]^{\beta}
$$
where using the properties of the logarithms we get,
$$
\ln r = \ln k + \alpha \ln [NO] + \beta \ln [H_2]
$$
We can solve this system of linear equations to find the values of $\ln k$, $\alpha$, and $\beta$.

In [ ]:
X = np.array([[0.0050,0.002],
              [0.01,0.002],
              [0.01,0.004]])
X = np.log(X)
X = np.column_stack((np.ones(3,), X))  # why the np.ones(3,)?

y = np.array([1.25E-5,5E-5, 1E-4])
y = np.log(y)

# magic
w = np.linalg.solve(X,y)


ln_k = w[0]
alpha = w[1]
beta = w[2]
print(f"ln(k) = {ln_k:.2f}", f"k = {np.exp(ln_k):.2f}")
print(f"alpha = {alpha:.2f}")
print(f"beta = {beta:.2f}")

# Steady State Solution using gradient descent
Let's consider the following reaction,
$$
2NO + Br_2 \to 2NOBr
$$
with the forward and reverse equilibrium constants ($k_f$ and $k_r$).

We can derive three different rate laws for each species,
$$
\frac{d [NO]}{dt} = 2k_b[NOBr]^2 - 2k_f[Br_2][NO]^2\\

\frac{d [Br_2]}{dt} = k_b[NOBr]^2 - k_f[Br_2][NO]^2\\

\frac{d [NOBr]}{dt} = 2k_f[Br_2][NO]^2 - 2k_f[NOBr]^2
$$

The goal is to find the values of $[NO], [Br_2]$ and $[NOBr]$,  where $\frac{d [NO]}{dt} = \frac{d [Br_2]}{dt} = \frac{d [NOBr]}{dt} = 0$.<br>
For these conditions we can use gradient descent to find these steady state concentrations.

In [ ]:
def equations_of_motion(x, kf, kb):
    rf = kf * x[0]**2 * x[1]
    rb = kb * x[2]**2
    return [2*(rb - rf), rb - rf, 2*(rf - rb)]

def gradient_of_equations_of_motion(x, kf, kb):
    dg1_dx1 = -4*kf*x[1]*x[0]
    dg2_dx1 = -2*kf*x[1]*x[0]
    dg3_dx1 = 2*kf*x[1]*x[0]
    
    dg1_dx2 = -2*kf*x[2]**2
    dg2_dx2 = -kf*x[2]**2
    dg3_dx2 = 2*kf*x[2]**2
    
    dg1_dx3 = 4*kb*x[2]
    dg2_dx3 = 2*kb*x[2]
    dg3_dx3 = -4*kb*x[2]
    
    jac = np.array([[dg1_dx1, dg1_dx2, dg1_dx3], 
                    [dg2_dx1, dg2_dx2, dg2_dx3], 
                    [dg3_dx1, dg3_dx2, dg3_dx3]])
    return jac

def error_function(x, kf, kb):
    g = equations_of_motion(x,kf,kb)
    return 0.5 * np.dot(g, g)


def gradient_error_function(x, kf, kb):
    g = equations_of_motion(x, kf, kb)
    jac_g = gradient_of_equations_of_motion(x, kf, kb)
    return jac_g.T @ g

    

In [ ]:
x0 = np.ones(3)
kf, kb = 0.42, 0.17

In [ ]:
eta = 0.1
for i in range(15):
    dx0 = gradient_error_function(x0, kf, kb)
    e = error_function(x0, kf, kb)
    print(f'Itr {i+1}, e = ', e, 'x = ', x0, '|dx| = ', np.linalg.norm(dx0))
    x0 = x0 - eta * dx0

print('Steady State concentrations')
print(f'[NO] = {x0[0]:.3f}')
print(r'[Br2]'+ f' = {x0[1]:.3f}')
print(f'[NOBr] = {x0[2]:.3f}')